In [4]:
from ibapi.contract import Contract
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import psycopg2
import threading
from time import sleep
import pandas as pd
DB_HOST = 'localhost'
DB_NAME = 'us'
DB_USER = 'postgres'
DB_PASS = 'buddyrich134'
conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
cursor = conn.cursor()
cursor.execute("SELECT distinct code from public.maincode where listed = 'us30';")
conn.commit()
code_list = [i[0] for i in cursor.fetchall()]
if 'DIA' not in code_list:
    code_list = code_list + ['DIA']
print(len(code_list))
# try:
#     app.disconnect()
# except:
#     pass
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.df = []
        self.minute = -1
        self.count = 0
        self.req_id_to_contract = {}

    def realtimeBar(self, reqId, time, open_, high, low, close, volume, wap, count):
        timestamp_utc = pd.to_datetime(time, unit='s', utc=True)
        tz = 'America/New_York'
        da = timestamp_utc.tz_convert(tz).strftime("%Y-%m-%d %H:%M:%S")
        # print(close, da, reqId)
        code = code_list[reqId-1]
        print(time, code, close)
        if da[-2:] == '00':
            self.df.append([da, code, close, wap, volume])
            
def run_loop():
    app.run()

app = IBapi()
app.connect('127.0.0.1', 7496, 1111)
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
reqID = 110
for symbol in ['AAPL', "DIA"]:
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    if symbol == "CAT":
        contract.exchange = 'NYSE'

    app.reqRealTimeBars(reqId=reqID, contract=contract, barSize=5, whatToShow="MIDPOINT", useRTH=0, realTimeBarsOptions=[])
    reqID += 1

31


In [15]:
import pandas as pd
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from datetime import datetime
import threading
import time
from IPython.display import display, clear_output
import psycopg2
# Assuming app and backtest_code are already defined
class ContinuousProcessor:
    def __init__(self):
        self.index_da = None
        self.lock = threading.Lock()
        self.running = True
        self.reqId = 250
        
        self.conn = self.get_db_connection()
        self.cursor = self.conn.cursor()

        self.reqId += 1
    def get_db_connection(self):
        DB_HOST = 'localhost'
        DB_NAME = 'us'
        DB_USER = 'postgres'
        DB_PASS = 'buddyrich134'
        conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
        return conn
    def backtest_code(self, pivoted, code, rolling=5, benchmark_index='DIA'):
        return None
    def orderStatus(self, orderId, status: str, filled,
                         remaining, avgFillPrice: float, permId: int,
                         parentId: int, lastFillPrice: float, clientId: int,
                         whyHeld: str, mktCapPrice: float):
        super().orderStatus(orderId, status, filled, remaining,
                            avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice)
        print(orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, whyHeld)
    def continuous_loop(self):
        while self.running:
            clear_output(wait=True)
            if len(app.df) == 0: 
                print("market not open, app.df has len 0")
                print(pd.to_datetime(time.time()*1000000000))
                time.sleep(10)
                continue
            df = pd.DataFrame(app.df[-2000:], columns=['da', 'code', 'cl', 'wap', 'volume'])
            df = df.pivot(values='cl', columns='code', index='da').sort_index(ascending=True)
            index = df.index[-1]
            if len(df) >= 5 and self.index_da != index:
                self.index_da = index
                display(f'Running analysis for index: {self.index_da}')
                for code in df.columns:
                    if code == "DIA":
                        continue
                    self.backtest_code(df[['DIA', code]], code)
                ####
                time.sleep(30)
                pd.DataFrame(app.df[-2000:], columns=['da', 'code', 'cl', 'wap', 'volume']).to_parquet("US100.parquet")
                
                self.cursor.execute("SELECT da from public.stock_price order by da desc limit 1;")
                self.conn.commit()
                
                try:
                    da = self.cursor.fetchone()[0]
                    print(da)
                    insert_df = pd.DataFrame(app.df[-500:], columns=['da', 'code', 'cl', 'wap', 'volume'])
                    insert_df = insert_df[pd.to_datetime(insert_df['da']) > da]
                except:
                    insert_df = pd.DataFrame(app.df[:], columns=['da', 'code', 'cl', 'wap', 'volume'])
                self.cursor.executemany("INSERT INTO public.stock_price (da, code, cl, wap, vol) VALUES (%s, %s, %s, %s, %s);", insert_df.values.tolist())
                self.conn.commit()
            else:
                print(pd.to_datetime(time.time()*1000000000), f"len(data) = {len(df)}")
                time.sleep(10)
                ####

    def stop(self):
        self.running = False

processor = ContinuousProcessor()
loop_thread = threading.Thread(target=processor.continuous_loop, daemon=True)
loop_thread.start()

market not open, app.df has len 0
2024-09-09 10:47:55.837533952


In [14]:
app.df

[]